<h1><center> Data Cleaning File  <br><br> 
<font color='grey'> Cleaning Emissions and Traffic Data <br><br>


Emissions Data

In [ ]:
#setting libraries
import pandas as pd

In [ ]:
#reading in emissions data files
emissions_16 = pd.read_excel("2016_emissions.xlsx",
                            sheet_name = "Direct Emitters", #only picks first sheet
                            header = 3,
                            usecols = ["City", "State", "Zip Code", "County", "Latitude", "Longitude", "Industry Type (subparts)", "Industry Type (sectors)", "Total reported direct emissions"])
emissions_13 = pd.read_excel("2013_emissions.xlsx",
                            sheet_name = "Direct Emitters", #only picks first sheet
                            header = 3,
                            usecols = ["City", "State", "Zip Code", "County", "Latitude", "Longitude", "Industry Type (subparts)", "Industry Type (sectors)", "Total reported direct emissions"])
emissions_19 = pd.read_excel("2019_emissions.xlsx",
                            sheet_name = "Direct Emitters", #only picks first sheet
                            header = 3,
                            usecols = ["City", "State", "Zip Code", "County", "Latitude", "Longitude", "Industry Type (subparts)", "Industry Type (sectors)", "Total reported direct emissions"])

In [ ]:
#create column names
new_col_names = ["city", "state", "zipcode", "county", "latitude", "longitude", "industry_sub", "industry_main", "total_emissions"]

In [ ]:
#rename columns
for df in [emissions_16, emissions_13, emissions_19]:
    df.columns = new_col_names

In [ ]:
#save list of relevant states and industry
rel_states = ["DC", "MD", "VA"]
rel_indust = "MN|NN|C|W"

In [ ]:
#filter for dc, md, and va + relevant industries
emissions_16 = emissions_16.query('state in @rel_states & industry_sub.str.contains(@rel_indust, na = False)')
emissions_13 = emissions_13.query('state in @rel_states & industry_sub.str.contains(@rel_indust, na = False)')
emissions_19 = emissions_19.query('state in @rel_states & industry_sub.str.contains(@rel_indust, na = False)')

In [ ]:
#checking number of rows
[df.shape for df in [emissions_16, emissions_13, emissions_19]]

In [ ]:
#adding identifers before merging
emissions_13.loc[:, "time"] = 2013
emissions_16.loc[:, "time"] = 2016
emissions_19.loc[:, "time"] = 2019

In [ ]:
#merge
emissions = emissions_13.merge(emissions_16, how = "outer").merge(emissions_19, how = "outer")

In [ ]:
#convert to wide format
emissions = emissions.pivot_table(
    index = list(emissions.columns[:-2]),
    columns = 'time',
    values = 'total_emissions'
).reset_index()

In [ ]:
#set all colnames to str
emissions.columns = emissions.columns.astype(str)

In [ ]:
#filters for all data that has both 2013 and 2019
emissions = emissions[(~emissions['2013'].isna()) & (~emissions['2019'].isna())]

In [ ]:
emissions.shape

In [ ]:
emissions.query("state == 'DC'")

In [ ]:
emissions.query("state == 'MD'")

In [ ]:
emissions.query("state == 'VA'")

Traffic Data

In [8]:
#setting library
import requests
import os
import json
import pandas as pd
import numpy as np

In [9]:
def traffic_query(coord_list):
    """
    takes in a list of captial bikeshare coordinates
    to tell API what data to retrieve
    """
    
    #extract coordinates
    long = coord_list[0] #UPDATE LATER AFTER SEEING HOW IBADAT SET IT UP
    lat = coord_list[1]
    
    #run query
    B_URL = "https://gis.mwcog.org/wa/rest/services/RTDC/Traffic_Counts_Annual/MapServer/0/query?" #base url
    
    response = requests.get(
        B_URL,
        params = {
            "where": "1=1", #no filters
            "outFields": "STATION,COUNTY,AADT2013,AADT2016, AADT2019,XCOORD,YCOORD", #indicates which cols to return
            "geometry": f"{long},{lat}", #input coordinates
            "geometryType": "esriGeometryPoint", #indicates we're giving it points
            "distance": 500, #how far away from point
            "units": "esriSRUnit_Meter", #units in meters
            "inSR": "4326", #coordiante system
            "f": "json" #type of file to return
        }
    )
    
    #check if successful
    if response.status_code == 200:
        return response.json()
    else:
        return "Query Failed"

In [10]:
def clean_traffic(geo_json, item_len):
    """
    function to extract columns of interest from traffic jsons
    takes in the json created in traffic_query() as well as the length
    of response.json()["features"]
    """
    #storing
    temp = []
    
    for item in range(item_len):
        obs = {
                "id": geo_json["features"][item]["attributes"]["STATION"],
                "x": geo_json["features"][item]["attributes"]["XCOORD"],
                "y": geo_json["features"][item]["attributes"]["YCOORD"],
                "county": geo_json["features"][item]["attributes"]["COUNTY"],
                "2013": geo_json["features"][item]["attributes"]["AADT2013"],
                "2016": geo_json["features"][item]["attributes"]["AADT2016"],
                "2019": geo_json["features"][item]["attributes"]["AADT2019"]
            }
        temp.append(obs)    
        
    #return 
    return(temp)
    

In [11]:
#test coordinates 
coord_df = pd.DataFrame(columns=['cb_station', 'long', 'lat'])

coordinates = pd.DataFrame([
    {'cb_station': "a", 'long': -77.0334, 'lat': 38.89223},
    {'cb_station': "b", 'long': -77.1334, 'lat': 38.87223}
])

coord_df = pd.concat([coord_df, coordinates], ignore_index = True)


C:\Users\helen\AppData\Local\Temp\ipykernel_21996\1224742629.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  coord_df = pd.concat([coord_df, coordinates], ignore_index = True)


In [12]:
#create empty dataframe for storage
traffic_df = pd.DataFrame(columns=['station', 'change_AADT_mean', 'change_AADT_sem', 'long', 'lat'])

In [13]:
for xy in range(len(coord_df)):
    #runs query for coordinates
    test_json = traffic_query([coord_df["long"][xy], coord_df["lat"][xy]])
    
    #adds if statement in case query fails or returns no coordinates
    if test_json == "Query Failed" or len(test_json["features"]) == 0:
        #creates dataframe with NaN values for AADT
        append = pd.DataFrame(columns=['station', 'change_AADT_mean', 'change_AADT_sem', 'long', 'lat'])
        append.loc[len(append)] = np.nan
        append['station'] = coord_df["cb_station"][xy]
        append['long'] = coord_df["long"][xy]
        append['lat'] = coord_df["lat"][xy]
        #append it to traffic_df
        traffic_df = pd.concat([traffic_df, test_df], ignore_index = True)
        continue
        
    #cleans up resulting json
    test_clean = clean_traffic(test_json, len(test_json["features"]))
    #converts to pandas and filters for where there is data for both 2013 and 2019
    test_df = pd.DataFrame(test_clean)
    test_df = test_df[(~test_df['2013'].isna()) & (~test_df['2019'].isna())]
    
    #adds if-else statement 
    if len(test_df) == 0: #in case there is no row with data for both
        #creates dataframe with NaN values for AADT
        append = pd.DataFrame(columns=['station', 'change_AADT_mean', 'change_AADT_sem', 'long', 'lat'])
        append.loc[len(append)] = np.nan
        append['station'] = coord_df["cb_station"][xy]
        append['long'] = coord_df["long"][xy]
        append['lat'] = coord_df["lat"][xy]
        #append it to traffic_df
        traffic_df = pd.concat([traffic_df, test_df], ignore_index = True)
        continue
    elif len(test_df) == 1: #if there is only one row and we can't compute SEM
        test_df = (test_df.assign(change_AADT = test_df["2013"] - test_df["2019"]).
         filter(["change_AADT"]).
         agg(["mean", "sem"]).
         reset_index().
         pivot_table(
                index = None,
                columns = 'index',
                values = 'change_AADT').
         rename(columns = {"mean" : "change_AADT_mean",
                           "sem" : "change_AADT_sem"}).
         reset_index(drop=True)
         )
        test_df['change_AADT_sem'] = np.nan
    else:
        #enough rows to compute both mean change in traffic volume & standard error
        test_df = (test_df.assign(change_AADT = test_df["2013"] - test_df["2019"]).
         filter(["change_AADT"]).
         agg(["mean", "sem"]).
         reset_index().
         pivot_table(
                index = None,
                columns = 'index',
                values = 'change_AADT').
         rename(columns = {"mean" : "change_AADT_mean",
                           "sem" : "change_AADT_sem"}).
         reset_index(drop=True)
         )
    #adds station and search coordinates
    test_df['station'] = coord_df["cb_station"][xy]
    test_df['long'] = coord_df["long"][xy]
    test_df['lat'] = coord_df["lat"][xy]
    #reorder cols to match storage dataframe
    test_df = test_df[['station', 'change_AADT_mean', 'change_AADT_sem', 'long', 'lat']]
    #add to storage dataframe
    traffic_df = pd.concat([traffic_df, test_df], ignore_index = True)
    

C:\Users\helen\AppData\Local\Temp\ipykernel_21996\25788858.py:69: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  traffic_df = pd.concat([traffic_df, test_df], ignore_index = True)


In [14]:
traffic_df

,station,change_AADT_mean,change_AADT_sem,long,lat
0,a,-1825.4,1966.061756,-77.0334,38.89223
1,b,2000.0,NaN,-77.1334,38.87223
